In [ ]:
import os
import librosa
import IPython.display as ipd
import matplotlib.pyplot as plt
import numpy as np
from scipy.io import wavfile
import warnings
warnings.filterwarnings("ignore")

In [ ]:
#pip install librosa

In [ ]:
ls C:\Users\ma2747\Downloads\tensorflow-speech-recognition-challenge

In [ ]:
os.listdir(r'C:\Users\ma2747\Downloads\tensorflow-speech-recognition-challenge')

In [ ]:
#pip install os

In [ ]:
# Define the path to the audio file
train_audio_path = r'C:\Users\ma2747\Downloads\tensorflow-speech-recognition-challenge\train\audio'
file_path = os.path.join(train_audio_path, 'yes', '0a7c2a8d_nohash_0.wav')

# Load the audio file
samples, sample_rate = librosa.load(file_path, sr=16000)

# Plot the raw wave
fig = plt.figure(figsize=(14, 8))
ax1 = fig.add_subplot(211)
ax1.set_title('Raw wave of ' + file_path)
ax1.set_xlabel('Time')
ax1.set_ylabel('Amplitude')
ax1.plot(np.linspace(0, len(samples) / sample_rate, len(samples)), samples)
plt.show()


In [ ]:
ipd.Audio(samples, rate=sample_rate)

In [ ]:
print(sample_rate)

In [ ]:
new_sample_rate = 8000

# Resample the audio
samples_resampled = librosa.resample(samples, orig_sr=sample_rate, target_sr=new_sample_rate)

# Play the resampled audio
ipd.Audio(samples_resampled, rate=new_sample_rate)

In [ ]:
 labels=os.listdir(train_audio_path)

In [ ]:
#find count of each label and plot bar graph
no_of_recordings=[]
for label in labels:
    waves = [f for f in os.listdir(train_audio_path + '/'+ label) if f.endswith('.wav')]
    no_of_recordings.append(len(waves))
#plot
plt.figure(figsize=(30,5))
index = np.arange(len(labels))
plt.bar(index, no_of_recordings)
plt.xlabel('Commands', fontsize=12)
plt.ylabel('No of recordings', fontsize=12)
plt.xticks(index, labels, fontsize=15, rotation=60)
plt.title('No. of recordings for each command')
plt.show()

In [ ]:
labels=["yes", "no", "up", "down", "left", "right", "on", "off", "stop", "go"]

In [ ]:
duration_of_recordings=[]
for label in labels:
    waves = [f for f in os.listdir(train_audio_path + '/'+ label) if f.endswith('.wav')]
    for wav in waves:
        sample_rate, samples = wavfile.read(train_audio_path + '/' + label + '/' + wav)
        duration_of_recordings.append(float(len(samples)/sample_rate))
plt.hist(np.array(duration_of_recordings))

In [ ]:
import os
import librosa

train_audio_path = r'C:\Users\ma2747\Downloads\tensorflow-speech-recognition-challenge\train\audio'
all_wave = []
all_label = []

for label in labels:
    print(label)
    waves = [f for f in os.listdir(os.path.join(train_audio_path, label)) if f.endswith('.wav')]
    
    for wav in waves:
        samples, sample_rate = librosa.load(os.path.join(train_audio_path, label, wav), sr=16000)
        samples = librosa.resample(samples, orig_sr=sample_rate, target_sr=8000)
        
        if len(samples) == 8000:
            all_wave.append(samples)
            all_label.append(label)


In [ ]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
y=le.fit_transform(all_label)
classes= list(le.classes_)

In [ ]:
from tensorflow.keras import utils as utils

In [ ]:
y=utils.to_categorical(y, num_classes=len(labels))

In [ ]:
all_wave = np.array(all_wave).reshape(-1,8000,1)

In [ ]:
from sklearn.model_selection import train_test_split
x_tr, x_val, y_tr, y_val = train_test_split(np.array(all_wave),np.array(y),stratify=y,test_size = 0.2,random_state=777,shuffle=True)

In [ ]:
from tensorflow import keras as keras

In [ ]:
#Deep learning
import tensorflow.keras as keras
#from tensorflow.keras.layers import Dense, Dropout, Flatten, Conv1D
from tensorflow.keras import Input, layers
from tensorflow.keras import backend as K

In [ ]:
K.clear_session()

In [ ]:
inputs = Input(shape=(8000,1))
#First Conv1D layer
conv = layers.Conv1D(8,13, padding='valid', activation='relu',strides=1)(inputs)
conv = layers.MaxPooling1D(3)(conv)
conv = layers.Dropout(0.3)(conv)
#Second Conv1D layer
conv = layers.Conv1D(16, 11, padding='valid', activation='relu',strides=1)(conv)
conv = layers.MaxPooling1D(3)(conv)
conv = layers.Dropout(0.3)(conv)
#Third Conv1D layer
conv = layers.Conv1D(32, 9, padding='valid', activation='relu', strides=1)(conv)
conv = layers.MaxPooling1D(3)(conv)
conv = layers.Dropout(0.3)(conv)
#Fourth Conv1D layer
conv = layers.Conv1D(64, 7, padding='valid', activation='relu', strides=1)(conv)
conv = layers.MaxPooling1D(3)(conv)
conv = layers.Dropout(0.3)(conv)
#Flatten layer
conv = layers.Flatten()(conv)
#Dense Layer 1
conv = layers.Dense(256, activation='relu')(conv)
conv = layers.Dropout(0.3)(conv)
#Dense Layer 2
conv = layers.Dense(128, activation='relu')(conv)
conv = layers.Dropout(0.3)(conv)
outputs = layers.Dense(len(labels), activation='softmax')(conv)
model = keras.Model(inputs, outputs)
model.summary()


In [ ]:
model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])

In [ ]:
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

In [ ]:
from keras.callbacks import EarlyStopping, ModelCheckpoint

# Early stopping callback
es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=10, min_delta=0.0001)

# Model checkpoint callback
mc = ModelCheckpoint('best_model.h5', monitor='val_acc', verbose=1, save_best_only=True, mode='max')


In [ ]:
history=model.fit(x_tr, y_tr ,epochs=1, callbacks=[es,mc], batch_size=32,validation_data=(x_val,y_val))

In [ ]:
from matplotlib import pyplot
pyplot.plot(history.history['loss'], label='train')
pyplot.plot(history.history['val_loss'], label='test')
pyplot.legend()
pyplot.show()

In [ ]:
from keras.models import load_model
model=load_model('best_model.hdf5')

In [ ]:
def predict(audio):
    prob=model.predict(audio.reshape(1,8000,1))
    index=np.argmax(prob[0])
    return classes[index]

In [ ]:
import random
index=random.randint(0,len(x_val)-1)
samples=x_val[index].ravel()
print("Audio:",classes[np.argmax(y_val[index])])
ipd.Audio(samples, rate=8000)